## 2.6. Pattern matching com sequências

No livro: https://pythonfluente.com/#sequence_patterns_sec

In [1]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, [-23.547778, -46.635833]),
]

def report():
    print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
    for record in metro_areas:
        match record:  # (1)
            case [name, _, _, (lat, lon)] if lon <= 0:  # (2)
                print(f'{name:15} | {lat:9.4f} | {lon:9.4f}')

report()

                |  latitude | longitude
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
São Paulo       |  -23.5478 |  -46.6358


In [2]:
def report():
    print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
    for record in metro_areas:
        name, _, _, (lat, lon) = record
        if lon <= 0:
            print(f'{name:15} | {lat:9.4f} | {lon:9.4f}')

report()

                |  latitude | longitude
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
São Paulo       |  -23.5478 |  -46.6358


### Exemplo que não tem no livro

In [3]:
statement = """
Format Net Units Net Sales Royalty % Net Royalty
Book 277 8,389.06 10.0000 838.91
Foreign Rights 1 1,037.73 25.0000 259.43
License 1 16.36 10.0000 1.64
On-Line Access 1 2,520.50 10.0000 252.05
eBook 370 7,200.81 10.0000 720.08
Book Reserve Return From (MAY 2016 O'Reilly) 483.29
Book Reserve 20.0000 -167.78
"""

def parse_token(token: str) -> int | float | str:
    """Convert number-like token to int or float."""
    maybe_number = token.replace(',', '_')
    try:
        return int(maybe_number)
    except ValueError:
        try:
            return float(maybe_number)
        except ValueError:
            pass
    return token  # unchanged

parse_token('banana'), parse_token('42'), parse_token('3.14')

('banana', 42, 3.14)

In [4]:
def tokenize(line):
    return [parse_token(t) for t in line.split()]

for line in statement.strip().split('\n'):
    print(tokenize(line))

['Format', 'Net', 'Units', 'Net', 'Sales', 'Royalty', '%', 'Net', 'Royalty']
['Book', 277, 8389.06, 10.0, 838.91]
['Foreign', 'Rights', 1, 1037.73, 25.0, 259.43]
['License', 1, 16.36, 10.0, 1.64]
['On-Line', 'Access', 1, 2520.5, 10.0, 252.05]
['eBook', 370, 7200.81, 10.0, 720.08]
['Book', 'Reserve', 'Return', 'From', '(MAY', 2016, "O'Reilly)", 483.29]
['Book', 'Reserve', 20.0, -167.78]


In [5]:
from typing import NamedTuple

class LineItem(NamedTuple):
    description: str
    units: int
    sales: float
    roy_pct: float
    royalty: float

def eval_line(text):
    tokens = tokenize(text)
    match tokens:
        case [*words, int(units), float(sales), float(roy_pct), float(royalty)]:
            description = ' '.join(words)
            return LineItem(description, units, sales, roy_pct, royalty)
        case _:
            return '?: ' + repr(tokens)

for line in statement.strip().split('\n'):
    print(eval_line(line))

?: ['Format', 'Net', 'Units', 'Net', 'Sales', 'Royalty', '%', 'Net', 'Royalty']
LineItem(description='Book', units=277, sales=8389.06, roy_pct=10.0, royalty=838.91)
LineItem(description='Foreign Rights', units=1, sales=1037.73, roy_pct=25.0, royalty=259.43)
LineItem(description='License', units=1, sales=16.36, roy_pct=10.0, royalty=1.64)
LineItem(description='On-Line Access', units=1, sales=2520.5, roy_pct=10.0, royalty=252.05)
LineItem(description='eBook', units=370, sales=7200.81, roy_pct=10.0, royalty=720.08)
?: ['Book', 'Reserve', 'Return', 'From', '(MAY', 2016, "O'Reilly)", 483.29]
?: ['Book', 'Reserve', 20.0, -167.78]


In [6]:
class Adjustment(NamedTuple):
    description: str
    royalty: float

def eval_line(tokens):
    match tokens:
        case [*words, int(units), float(sales), float(roy_pct), float(royalty)]:
            description = '_'.join(words)
            return LineItem(description, units, sales, roy_pct, royalty)
        case [*words, float(royalty)]:
            description = '_'.join(str(w) for w in words)
            return Adjustment(description, royalty)
        case _:
            return '?: ' + repr(tokens)

for line in statement.strip().split('\n'):
    tokens = tokenize(line)
    print(eval_line(tokens))

?: ['Format', 'Net', 'Units', 'Net', 'Sales', 'Royalty', '%', 'Net', 'Royalty']
LineItem(description='Book', units=277, sales=8389.06, roy_pct=10.0, royalty=838.91)
LineItem(description='Foreign_Rights', units=1, sales=1037.73, roy_pct=25.0, royalty=259.43)
LineItem(description='License', units=1, sales=16.36, roy_pct=10.0, royalty=1.64)
LineItem(description='On-Line_Access', units=1, sales=2520.5, roy_pct=10.0, royalty=252.05)
LineItem(description='eBook', units=370, sales=7200.81, roy_pct=10.0, royalty=720.08)
Adjustment(description="Book_Reserve_Return_From_(MAY_2016_O'Reilly)", royalty=483.29)
Adjustment(description='Book_Reserve_20.0', royalty=-167.78)
